Let aussume that we need to make this computation

$\sqrt{|(a+b)\times c|}$

and we want to compute the standard deviation of the result assuming that a, b and c are normal distributed independent variables. Clearly the problem is quite naive but we want to address is as if we will need a cluster to solve it. 

We can partition the problem in a three conscutive operations
1. A sum: $(a+b)$
2. A multiplication of the result 1 with c: $(a+b)\times c$
3. A sqrt of the result of 2: $\sqrt{|(a+b)\times c|}$

In [1]:
import sys
sys.path.append('/home/jovyan/local_host_home/CERNBox/2021/make_tree')
import tree_maker
from tree_maker import NodeJob

In [2]:
import numpy as np
a=np.random.randn(4)
b=np.random.randn(4)
c=np.random.randn(2)

my_list_original=[]
for ii in c:
    my_list_original+=list(np.sqrt(np.abs((a+b)*ii)))
my_list_original=sorted(my_list_original)

In [3]:
#root
root = NodeJob(name='root', parent=None)
root.path = '/home/jovyan/local_host_home/CERNBox/2021/make_tree/examples/001_example'

#first generation
for node in root.leaves:
    node.children=[NodeJob(name=f"{child:03}",
                           parent=node,
                           path = f"{node.path}/{child:03}",
                           template_path = '/home/jovyan/local_host_home/CERNBox/2021/make_tree/examples/001_example/templates/sum_it',
                           dictionary={'a':float(a[child]), 'b':float(b[child])})
                   for child in range(len(a))]

#second generation
for node in root.leaves:
    node.children=[NodeJob(name=f"{child:03}",
                           parent=node,
                           path = f"{node.path}/{child:03}",
                           template_path = '/home/jovyan/local_host_home/CERNBox/2021/make_tree/examples/001_example/templates/multiply_it',
                           dictionary={'c':float(c[child])})
                   for child in range(len(c))]
    
#third generation
for node in root.leaves:
    node.children=[NodeJob(name=f"{child:03}",
                           parent=node, 
                           path = f"{node.path}/{child:03}",
                           template_path = '/home/jovyan/local_host_home/CERNBox/2021/make_tree/examples/001_example/templates/square_root_it',
                           dictionary={'a':float(c[child])})
                           for child in range(1)]

In [4]:
root.print_it()

root
├── 000
│   ├── 000
│   │   ╰── 000
│   ╰── 001
│       ╰── 000
├── 001
│   ├── 000
│   │   ╰── 000
│   ╰── 001
│       ╰── 000
├── 002
│   ├── 000
│   │   ╰── 000
│   ╰── 001
│       ╰── 000
╰── 003
    ├── 000
    │   ╰── 000
    ╰── 001
        ╰── 000


In [5]:
# save the tree
root.to_yaml()

In [6]:
# Load the tree from a yaml
root=tree_maker.tree_from_yaml('./node.yaml')
root.print_it()

root
├── 000
│   ├── 000
│   │   ╰── 000
│   ╰── 001
│       ╰── 000
├── 001
│   ├── 000
│   │   ╰── 000
│   ╰── 001
│       ╰── 000
├── 002
│   ├── 000
│   │   ╰── 000
│   ╰── 001
│       ╰── 000
╰── 003
    ├── 000
    │   ╰── 000
    ╰── 001
        ╰── 000


In [7]:
root.children[0].dictionary

{'a': 0.48617312066239876, 'b': 1.129114462645723}

In [8]:
# STEP 1 cloning
root.rm_children()
[x.clone_children() for x in root.generation(0)]
[x.clone_children() for x in root.generation(1)]
[x.clone_children() for x in root.generation(2)];

In [9]:
def run_1(self):
    import os
    print('It is new!')
    os.system(f'cd {self.path}; python run.py;')

NodeJob.run_1=run_1

for node in root.generation(1):
    node.mutate()
    node.run_1()

It is new!
It is new!
It is new!
It is new!


In [14]:
for node in root.generation(2):
    parent_output = tree_maker.from_yaml(node.parent.path+'/output.yaml')
    node.dictionary['sum_a_b']=parent_output['result']
    node.mutate()
    node.run()

In [15]:
for node in root.generation(3):
    parent_output = tree_maker.from_yaml(node.parent.path+'/output.yaml')
    node.dictionary['a']=parent_output['result']
    node.mutate()
    node.run()

In [16]:
my_list=[]
for node in root.generation(3):
    output = tree_maker.from_yaml(node.path+'/output.yaml')
    my_list.append(output['result'])

In [17]:
np.array(sorted(my_list))-np.array(my_list_original)

array([0., 0., 0., 0., 0., 0., 0., 0.])